In [1]:
import geopandas as gpd

from pathlib import Path
from geowrangler.datasets import geofabrik
from geowrangler import distance_zonal_stats as dzs
from geowrangler import vector_zonal_stats as vzs

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


### Set-up Input Parameters

In [2]:
DATA_DIR = Path("../../data/")
ADMIN_FPATH = DATA_DIR / "admin-bounds"

In [3]:
admin_bounds_gdf = gpd.read_file(ADMIN_FPATH / "target_admin_bounds.shp")
admin_bounds_gdf.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."


In [4]:
admin_bounds_gdf["ADM1_EN"].unique()

array(['Region I', 'Region III', 'Region V', 'Region VI', 'Region VII',
       'Region VIII', 'Region IX', 'Region X', 'Region XI',
       'National Capital Region'], dtype=object)

### Utils

In [14]:
def add_osm_poi_features(
    aoi,
    country,
    year,
    osm_data_manager,
    use_cache=True,
    poi_types=NONHEALTH_CRITICAL,
    metric_crs="epsg:3857",
    inplace=False,
    nearest_poi_max_distance=10000,
):
    """Generates features for the AOI based on OSM POI data (POIs, roads, etc)."""

    # Load-in the OSM POIs data
    osm = osm_data_manager.load_pois(country, year=year, use_cache=use_cache)

    # Create a copy of the AOI gdf if not inplace to avoid modifying the original gdf
    if not inplace:
        aoi = aoi.copy()

    # GeoWrangler: Count number of all POIs per tile
    aoi = vzs.create_zonal_stats(
        aoi,
        osm,
        overlap_method="intersects",
        aggregations=[{"func": "count", "output": "poi_count", "fillna": True}],
    )

    # Count specific aoi types
    for poi_type in poi_types:
        # GeoWrangler: Count with vector zonal stats
        aoi = vzs.create_zonal_stats(
            aoi,
            osm[osm["fclass"] == poi_type],
            overlap_method="intersects",
            aggregations=[
                {"func": "count", "output": f"{poi_type}_count", "fillna": True}
            ],
        )

        # GeoWrangler: Distance with distance zonal stats
        col_name = f"{poi_type}_nearest"
        aoi = dzs.create_distance_zonal_stats(
            aoi.to_crs(metric_crs),
            osm[osm["fclass"] == poi_type].to_crs(metric_crs),
            max_distance=nearest_poi_max_distance,
            aggregations=[],
            distance_col=col_name,
        ).to_crs("epsg:4326")

        # If no POI was found within the distance limit, set the distance to the max distance
        aoi[col_name] = aoi[col_name].fillna(value=nearest_poi_max_distance)

    return aoi

## Get OSM Data for PH

In [7]:
osm = geofabrik.OsmDataManager(cache_dir="../../data/osm/")

In [8]:
osm_pois = osm.load_pois(region="philippines", year="2020", use_cache=True)
osm_pois.head(2)

2023-03-29 17:24:21.561 | INFO     | geowrangler.datasets.geofabrik:download_osm_region_data:128 - OSM Data: Cached data available for philippines at ../../data/osm/osm/philippines-200101-free.shp.zip? True
2023-03-29 17:24:21.562 | DEBUG    | geowrangler.datasets.geofabrik:load_pois:222 - OSM POIs for philippines and year 2020 being loaded from ../../data/osm/osm/philippines-200101-free.shp.zip


,osm_id,code,fclass,name,geometry
0,14401658,2742,viewpoint,East Point,POINT (120.62020 14.38548)
1,14446500,2721,attraction,Intramuros,POINT (120.97533 14.59059)


## Filter to needed POI Types

In [9]:
fclass_list = osm_pois["fclass"].unique().tolist()

with open(DATA_DIR / "osm/pois_fclass.txt", "w") as fp:
    for item in fclass_list:
        fp.write("%s\n" % item)
    print("Done")

Done


In [10]:
# check how many are the fclasses
len(fclass_list)

132

In [ ]:
# add health tags too
# make sure to add sanitation tags, water waste tags
# get the osm_water
# explore making a similar functino for the polygons


# can explore adding it to the OSM (low prio)
NONHEALTH_CRITICAL = [
    "fire_station",
    "school",
    "college",
    "bank",
    "supermarket",
    "telephone",
    "public_building",
    "park",
    "university",
    "police",
    "tower",
    "toilet",
    "water_tower",
    "lighthouse",
    "convenience",
    "kindergarten",
    "town_hall",
    "community_centre",
    "comms_tower",
    "atm",
    "drinking_water",
    "market_place",
]

## Get OSM POI features for target areas

**Reminder:** Do this for 20 years worth of data. 

In [1]:
gdf = add_osm_poi_features(
    admin_bounds_gdf,
    "philippines",
    "2020",
    osm,
    use_cache=True,
    poi_types=NONHEALTH_CRITICAL,
)

NameError: name 'add_osm_poi_features' is not defined

In [19]:
remove = ["ADM1_PCODE", "ADM2_PCODE", "ADM3_PCODE", "ADM4_PCODE"]
check = gdf.loc[:, ~gdf.columns.isin(remove)]
check

,ADM1_EN,ADM2_EN,ADM3_EN,ADM4_EN,geometry,poi_count,fire_station_count,fire_station_nearest,school_count,school_nearest,...,community_centre_count,community_centre_nearest,comms_tower_count,comms_tower_nearest,atm_count,atm_nearest,drinking_water_count,drinking_water_nearest,market_place_count,market_place_nearest
0,Region I,Pangasinan,Dagupan City,Lomboy,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",0.0,0.0,2152.886539,0.0,1051.855382,...,0.0,8601.790815,0.0,1173.590976,0.0,476.327303,0.0,10000.000000,0.0,10000.0
1,Region I,Pangasinan,Dagupan City,Tapuac,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",47.0,0.0,931.181252,0.0,77.380080,...,0.0,8644.874045,0.0,2471.441139,0.0,1970.587480,0.0,10000.000000,0.0,10000.0
2,Region I,Pangasinan,Dagupan City,Pantal,"POLYGON ((120.34737 16.06009, 120.34761 16.060...",82.0,0.0,18.102258,1.0,0.000000,...,0.0,6254.871168,0.0,168.559995,0.0,1900.977424,0.0,10000.000000,0.0,10000.0
3,Region I,Pangasinan,Dagupan City,Barangay I (T. Bugallon),"POLYGON ((120.34054 16.04489, 120.34054 16.044...",51.0,0.0,395.419002,0.0,306.521899,...,0.0,7732.743800,0.0,1689.273010,0.0,2717.501600,0.0,10000.000000,0.0,10000.0
4,Region III,Nueva Ecija,Palayan City,Imelda Valley,"POLYGON ((121.12250 15.58028, 121.12687 15.579...",1.0,0.0,10000.000000,0.0,2152.734302,...,0.0,1683.508479,0.0,10000.000000,0.0,9912.654323,0.0,10000.000000,0.0,10000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,National Capital Region,"NCR, Second District",City of Mandaluyong,Namayan,"POLYGON ((121.02328 14.58135, 121.02309 14.581...",4.0,0.0,703.481685,0.0,508.501098,...,0.0,1116.157991,0.0,1727.607708,0.0,595.927985,0.0,2310.684732,0.0,10000.0
875,National Capital Region,"NCR, Second District",City of Mandaluyong,Plainview,"POLYGON ((121.03657 14.57933, 121.03734 14.579...",83.0,1.0,0.000000,0.0,379.327836,...,0.0,434.914649,0.0,624.581514,1.0,0.000000,0.0,2460.097735,0.0,10000.0
876,National Capital Region,"NCR, Third District",City of Navotas,Navotas West,"MULTIPOLYGON (((120.94767 14.65256, 120.94768 ...",0.0,0.0,458.785698,0.0,439.465322,...,0.0,943.118141,0.0,4133.971332,0.0,430.185952,0.0,5680.485435,0.0,10000.0
877,National Capital Region,"NCR, Third District",City of Navotas,Tanza,"POLYGON ((120.91841 14.71296, 120.92356 14.708...",3.0,0.0,292.784202,1.0,0.000000,...,0.0,1785.128453,0.0,1329.180524,0.0,1891.836984,0.0,7612.476342,0.0,10000.0


In [21]:
check.sort_values(by="poi_count", ascending=False)

,ADM1_EN,ADM2_EN,ADM3_EN,ADM4_EN,geometry,poi_count,fire_station_count,fire_station_nearest,school_count,school_nearest,...,community_centre_count,community_centre_nearest,comms_tower_count,comms_tower_nearest,atm_count,atm_nearest,drinking_water_count,drinking_water_nearest,market_place_count,market_place_nearest
137,National Capital Region,"NCR, Fourth District",City of Muntinlupa,Alabang,"POLYGON ((121.05364 14.42508, 121.05394 14.424...",356.0,1.0,0.000000,2.0,0.000000,...,0.0,951.609018,0.0,1702.531920,3.0,0.000000,0.0,10000.000000,0.0,10000.0
441,National Capital Region,"NCR, Fourth District",City of Muntinlupa,New Alabang Village,"POLYGON ((121.03128 14.42464, 121.03163 14.424...",198.0,0.0,1012.251495,2.0,0.000000,...,0.0,242.938229,0.0,41.410863,4.0,0.000000,0.0,8347.033282,0.0,10000.0
709,Region XI,Davao del Sur,Davao City,Bucana,"POLYGON ((125.60493 7.06428, 125.60548 7.06383...",187.0,1.0,0.000000,8.0,0.000000,...,1.0,0.000000,0.0,2499.164234,2.0,0.000000,1.0,0.000000,0.0,10000.0
265,National Capital Region,"NCR, Second District",City of Mandaluyong,Highway Hills,"POLYGON ((121.05614 14.57903, 121.05712 14.578...",176.0,0.0,217.083664,1.0,0.000000,...,0.0,522.913120,0.0,45.141442,3.0,0.000000,0.0,3040.152914,0.0,10000.0
869,Region XI,Davao del Sur,Davao City,Matina Crossing,"POLYGON ((125.59555 7.06240, 125.59667 7.06266...",164.0,1.0,0.000000,7.0,0.000000,...,6.0,0.000000,0.0,4011.852249,12.0,0.000000,0.0,1269.833400,0.0,10000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,Region VIII,Leyte,Tacloban City,Barangay 69,"MULTIPOLYGON (((124.98824 11.26014, 124.98824 ...",0.0,0.0,1362.726230,0.0,1319.930160,...,0.0,1420.872146,0.0,1940.166689,0.0,1279.992982,0.0,10000.000000,0.0,10000.0
511,Region VIII,Leyte,Tacloban City,Barangay 42-A,"POLYGON ((124.99739 11.24276, 124.99746 11.242...",0.0,0.0,924.087316,0.0,111.231390,...,0.0,128.669112,0.0,1516.948264,0.0,345.919185,0.0,9218.443127,0.0,10000.0
519,Region VIII,Leyte,Tacloban City,Barangay 6-A,"POLYGON ((125.00793 11.24576, 125.00793 11.245...",0.0,0.0,357.754221,0.0,68.464516,...,0.0,1275.420411,0.0,2553.197019,0.0,275.210018,0.0,9713.453016,0.0,10000.0
524,Region IX,Zamboanga del Sur,Zamboanga City,Cacao,"POLYGON ((122.14782 7.03953, 122.14966 7.04149...",0.0,0.0,10000.000000,0.0,20.729608,...,0.0,10000.000000,0.0,10000.000000,0.0,2086.352679,0.0,10000.000000,0.0,10000.0
